In [1]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import KFold, train_test_split
from sklearn import *
from scipy import *

from scipy.stats import skew
from scipy.stats.stats import pearsonr
from scipy.optimize import minimize_scalar

from hyperopt import hp, tpe
from hyperopt.fmin import fmin

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer

import re

# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;

from sklearn.decomposition import TruncatedSVD

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\ProgramData\Anacon

In [2]:
from xgboost import XGBClassifier

from sklearn import datasets
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import confusion_matrix

iris = datasets.load_iris()
X = iris.data
y = iris.target

clf = XGBClassifier()
pred = cross_val_predict(clf, X, y)
cm = confusion_matrix(y, pred)
print(cm)

[[50  0  0]
 [ 0 47  3]
 [ 0  2 48]]


In [123]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [73]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [124]:
all_data = pd.concat((train.loc[:,'ps_ind_01':'ps_calc_20_bin'],
                      test.loc[:,'ps_ind_01':'ps_calc_20_bin']))

In [93]:
col = [
        'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03',
       'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_15',
       'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01',
       'ps_reg_02', 
       'ps_car_01_cat', 'ps_car_02_cat',
       'ps_car_04_cat', 'ps_car_06_cat',
       'ps_car_08_cat', 'ps_car_09_cat', 
       'ps_car_11_cat', 'ps_car_11', 'ps_car_12', 'ps_car_13', 
       'ps_car_15', 
       ]

#### deleded columns
'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_14'
       'ps_car_10_cat'
       'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04',
       'ps_reg_03', 'ps_car_05_cat', 'ps_car_07_cat', 'ps_car_14', 'ps_car_03_cat', 
       'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
       'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
       'ps_calc_19_bin', 'ps_calc_20_bin'

In [94]:
all_data = all_data[col]

In [95]:
dtl = []
for i in col:
    dt = all_data[all_data[i]==-1]
    tup = (i,dt.shape)
    dtl.append(tup)

In [96]:
dtl

[('ps_ind_01', (0, 26)),
 ('ps_ind_02_cat', (523, 26)),
 ('ps_ind_03', (0, 26)),
 ('ps_ind_04_cat', (228, 26)),
 ('ps_ind_05_cat', (14519, 26)),
 ('ps_ind_06_bin', (0, 26)),
 ('ps_ind_07_bin', (0, 26)),
 ('ps_ind_08_bin', (0, 26)),
 ('ps_ind_09_bin', (0, 26)),
 ('ps_ind_15', (0, 26)),
 ('ps_ind_16_bin', (0, 26)),
 ('ps_ind_17_bin', (0, 26)),
 ('ps_ind_18_bin', (0, 26)),
 ('ps_reg_01', (0, 26)),
 ('ps_reg_02', (0, 26)),
 ('ps_car_01_cat', (267, 26)),
 ('ps_car_02_cat', (10, 26)),
 ('ps_car_04_cat', (0, 26)),
 ('ps_car_06_cat', (0, 26)),
 ('ps_car_08_cat', (0, 26)),
 ('ps_car_09_cat', (1446, 26)),
 ('ps_car_11_cat', (0, 26)),
 ('ps_car_11', (6, 26)),
 ('ps_car_12', (1, 26)),
 ('ps_car_13', (0, 26)),
 ('ps_car_15', (0, 26))]

In [97]:
all_data.dtypes

ps_ind_01          int64
ps_ind_02_cat      int64
ps_ind_03          int64
ps_ind_04_cat      int64
ps_ind_05_cat      int64
ps_ind_06_bin      int64
ps_ind_07_bin      int64
ps_ind_08_bin      int64
ps_ind_09_bin      int64
ps_ind_15          int64
ps_ind_16_bin      int64
ps_ind_17_bin      int64
ps_ind_18_bin      int64
ps_reg_01        float64
ps_reg_02        float64
ps_car_01_cat      int64
ps_car_02_cat      int64
ps_car_04_cat      int64
ps_car_06_cat      int64
ps_car_08_cat      int64
ps_car_09_cat      int64
ps_car_11_cat      int64
ps_car_11          int64
ps_car_12        float64
ps_car_13        float64
ps_car_15        float64
dtype: object

In [116]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
np.log1p(all_data[skewed_feats])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: divide by zero encountered in log1p


,ps_ind_01,ps_ind_02_cat,ps_ind_05_cat,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,...,ps_ind_18_bin,ps_reg_02,ps_car_03_cat,ps_car_04_cat,ps_car_12,ps_car_13,ps_calc_12,ps_calc_15_bin,ps_calc_18_bin,ps_calc_20_bin
0,1.098612,1.098612,0.000000,0.693147,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.182322,-inf,0.000000,0.336472,0.633227,0.693147,0.000000,0.000000,0.693147
1,0.693147,0.693147,0.000000,0.000000,0.693147,0.000000,0.0,0.0,0.0,0.0,...,0.693147,0.336472,-inf,0.000000,0.274770,0.481695,0.693147,0.000000,0.000000,0.000000
2,1.791759,1.609438,0.000000,0.000000,0.693147,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,-inf,0.000000,0.274770,0.495663,1.098612,0.000000,0.000000,0.000000
3,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.182322,0.000000,0.000000,0.317847,0.433695,1.098612,0.000000,0.000000,0.000000
4,0.000000,1.098612,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.470004,-inf,0.000000,0.274650,0.448417,0.693147,0.000000,0.693147,0.000000
5,1.791759,0.693147,0.000000,0.000000,0.000000,0.693147,0.0,0.0,0.0,0.0,...,0.000000,1.029619,-inf,0.000000,0.368789,0.630766,1.098612,0.000000,0.693147,0.693147
6,1.098612,0.693147,0.000000,0.693147,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.095310,-inf,0.000000,0.274770,0.494503,0.000000,0.000000,0.000000,0.000000
7,1.791759,0.693147,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.336472,0.000000,0.000000,0.369485,0.642156,0.693147,0.693147,0.000000,0.000000
8,1.791759,0.693147,0.000000,0.000000,0.693147,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.530628,-inf,0.000000,0.336472,0.576974,1.098612,0.000000,0.000000,0.693147
9,0.693147,0.693147,0.000000,0.693147,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.693147,0.875469,0.000000,0.693147,0.369640,0.899423,1.791759,0.000000,0.000000,0.000000


In [125]:
#log transform skewed numeric features:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: RuntimeWarning: divide by zero encountered in log1p


In [118]:
all_data = all_data.drop('ps_car_03_cat')

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [126]:
all_data = pd.get_dummies(all_data)

In [127]:
all_data = all_data.fillna(all_data.mean())

In [ ]:
svd = TruncatedSVD(n_components=30, n_iter=7, random_state=42)
svd.fit(all_data)  

In [ ]:
all_data = svd.transform(all_data)

In [128]:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y_train = train['target']

In [103]:
# LightGBM parameters
denom = 0
fold = 5 #Change to 5, 1 for Kaggle Limits
for i in range(fold):
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': {'auc'},
        #'num_class': 9,
        'learning_rate': 0.01,
        'max_depth': 28,
        'num_leaves': 136,
        'min_data_in_leaf': 8,
        'lambda_l1': 0.02,
        'num_iteration': 2000,
        'feature_fraction': 0.8, 
        'bagging_fraction': 0.8, 
        'bagging_freq': 5,
        #'colsample_bytree': 0.3542618105439753
        #'verbose': 0
    }
    X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_train, y_train, test_size=0.18, random_state=i)
        
    lgb_train = lgb.Dataset(X_train_cv, y_train_cv)
    lgb_eval = lgb.Dataset(X_test_cv, y_test_cv, reference=lgb_train)

# train
    gbm = lgb.train(params,
            lgb_train,
            #num_boost_round=rnds,
            valid_sets=lgb_eval,
            verbose_eval=10,
            early_stopping_rounds=20)
    
    if denom != 0:
        pred = gbm.predict(X_test, num_iteration=gbm.best_iteration+50)      
        preds += pred
    else:
        pred = gbm.predict(X_test, num_iteration=gbm.best_iteration+50)
        preds = pred.copy()
    denom += 1
    #submission = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])
    #submission = submission.reset_index()
    #submission.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]
    #submission['ID'] = pid
    #submission.to_csv('submission_xgb_fold_'  + str(i) + '.csv', index=False)
    
preds /= denom
submission = pd.read_csv("sample_submission.csv")
submission['target'] = preds
submission.columns = ["id","target"]
submission.to_csv("sub_LGBM.csv", index=False)
#submission['ID'] = pid
#y_pred = gbm.predict(X_test_vect_lgbm, num_iteration=gbm.best_iteration)
#y_pred = np.argmax(y_pred, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:100: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 20 rounds.
[10]	valid_0's auc: 0.621893
[20]	valid_0's auc: 0.623707
[30]	valid_0's auc: 0.624759
[40]	valid_0's auc: 0.625483
[50]	valid_0's auc: 0.626222
[60]	valid_0's auc: 0.626187
[70]	valid_0's auc: 0.626403
[80]	valid_0's auc: 0.627166
[90]	valid_0's auc: 0.627549


KeyboardInterrupt: 

In [104]:
def gini(truth, predictions):
    g = np.asarray(np.c_[truth, predictions, np.arange(len(truth)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(truth) + 1) / 2.
    return gs / len(truth)

def gini_xgb(predictions, truth):
    truth = truth.get_label()
    return 'gini', -1.0 * gini(truth, predictions) / gini(truth, truth)

def gini_lgb(truth, predictions):
    score = gini(truth, predictions) / gini(truth, truth)
    return 'gini', score, True

def gini_sklearn(truth, predictions):
    return gini(truth, predictions) / gini(truth, truth)

gini_scorer = make_scorer(gini_sklearn, greater_is_better=True, needs_proba=True)

In [107]:
def objective(params):
    params = {
        'num_leaves': int(params['num_leaves']),
        #'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'max_depth': int(params['max_depth']),
        'min_data_in_leaf': int(params['min_data_in_leaf']),
        'lambda_l1': '{:.3f}'.format(params['lambda_l1']),
    }
    
    clf = lgb.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.01,
        **params
    )
    
    score = cross_val_score(clf, X_train, y_train, scoring=gini_scorer, cv=StratifiedKFold(5)).mean()
    print("Gini {:.3f} params {}".format(score, params))
    return score

space = {
    'num_leaves': hp.quniform('num_leaves', 30, 150, 2),
    #'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'max_depth': hp.quniform('max_depth', 5, 30, 2),
    'min_data_in_leaf': hp.quniform('min_data_in_leaf', 1, 20, 1),
    'lambda_l1': hp.uniform('lambda_l1', 0.01, 0.05),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

Gini -0.267 params {'num_leaves': 60, 'max_depth': 6, 'min_data_in_leaf': 6, 'lambda_l1': '0.026'}


KeyboardInterrupt: 

In [ ]:
print("Hyperopt estimated optimum {}".format(best))

### LGBM optimum v1
Hyperopt estimated optimum {'colsample_bytree': 0.3542618105439753, 'lambda_l1': 0.020025402620244666, 'max_depth': 28.0, 'min_data_in_leaf': 8.0, 'num_leaves': 136.0}

### Xgboost optimization

In [ ]:
def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    }
    
    clf = xgb.XGBClassifier(
        n_estimators=250,
        learning_rate=0.05,
        nthread=4,
        **params
    )
    
    score = cross_val_score(clf, X_train, y_train, scoring=gini_scorer, cv=StratifiedKFold(5)).mean()
    print("Gini {:.3f} params {}".format(score, params))
    return(score)

space = {
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

In [ ]:
print("Hyperopt estimated optimum {}".format(best))

### xgboost optimum v1
Hyperopt estimated optimum {'colsample_bytree': 0.42220114349224486, 'gamma': 0.22144103013844213, 'max_depth': 6.0}

In [ ]:
def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']), 
        'max_depth': int(params['max_depth']),
    }
    clf = RandomForestClassifier(
        n_jobs=4, 
        class_weight='balanced', 
        **params
    )
    
    score = cross_val_score(clf, X_train, y_train, scoring=gini_scorer, cv=StratifiedKFold()).mean()
    
    print("Gini {:.3f} params {}".format(score, params))
    
    return(score)

space = {
    'n_estimators': hp.quniform('n_estimators', 25, 500, 25),
    'max_depth': hp.quniform('max_depth', 1, 10, 1)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

## Stucking

In [81]:
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
#ntrain = 1000
#ntest = 1000
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, X_train, y_train):
        self.clf.fit(X_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

class XgbWrapper2(object):
    def __init__(self, clf, seed=0, params=None):
        params['seed'] = seed
        self.clf = clf(**params)

    def train(self, X_train, y_train):
        self.clf.fit(X_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
        
class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, X_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

class LgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.n_estimators = params.pop('n_estimators', 500)

    def train(self, X_train, y_train):
        lgb_train = lgb.Dataset(X_train, y_train)
        self.gbdt = lgb.train(self.param, lgb_train, self.n_estimators)

    def predict(self, x):
        return self.gbdt.predict(x)
    
# Class to extend XGboost classifer

In [82]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)[:,1]
        oof_test_skf[i, :] = clf.predict(x_test)[:,1]

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [83]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

#Xgboosting parameters
xgb_params = {
    'min_child_weight': 10.0,
    'objective': 'binary:logistic',
    'max_depth': 7,
    'max_delta_step': 1.8,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'eta': 0.025,
    'gamma': 0.65,
    'num_boost_round' : 700
}

# Light GBM parameters
lgb_params = {
    'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': {'auc'},
        #'num_class': 9,
        'learning_rate': 0.01,
        'max_depth': 28,
        'num_leaves': 136,
        'min_data_in_leaf': 8,
        'lambda_l1': 0.02,
        'num_iteration': 2000,
        'feature_fraction': 0.8, 
        'bagging_fraction': 0.8, 
        'bagging_freq': 5,
        #'colsample_bytree': 0.3542618105439753
    }

In [12]:
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)
xg = XgbWrapper2(clf=xgb.XGBClassifier, seed=SEED, params=xgb_params)
lg = XgbWrapper2(clf=lgb.LGBMClassifier, seed=SEED, params=lgb_params)
#xg = XgbWrapper(seed=SEED, params=xgb_params) 
#lg = LgbWrapper(seed=SEED, params=lgb_params)

In [16]:
y_train = y_train.ravel()
x_train = X_train.values # Creates an array of the train data
x_test = X_test.values # Creats an array of the test data

In [85]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost
#svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier
xg_oof_train, xg_oof_test = get_oof(xg,x_train, y_train, x_test) # xgboost Classifier
lg_oof_train, lg_oof_test = get_oof(lg,x_train, y_train, x_test) # LightGBM Classifier

print("Training is complete")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:303: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:100: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:100: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:100: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:100: User

Training is complete


In [86]:
x_train = np.concatenate((et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, xg_oof_train, lg_oof_train), axis=1)
x_test = np.concatenate((et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, xg_oof_test, lg_oof_test), axis=1)

In [129]:
lg.feature_importances(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:100: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[ 6916  2439 10009  1912  3718  1444  1664  1014   982     5    14   141
    21   177 10657  1587  1567   693  7187  8825 19999  7036   851  2428
  1815  2682  7255  1533   631  3010   179 11870  2926  5964 19615 16080
  6897  8216  8227  8576  4671  4825  4960  5675  5670  4945  8866  7417
  4764  6144  8612   671  1332  1654  1116  1285   631]


In [89]:
# LightGBM parameters
denom = 0
fold = 5 #Change to 5, 1 for Kaggle Limits
for i in range(fold):
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': {'auc'},
        #'num_class': 9,
        'learning_rate': 0.01,
        'max_depth': 28,
        'num_leaves': 136,
        'min_data_in_leaf': 8,
        'lambda_l1': 0.02,
        'num_iteration': 2000,
        'feature_fraction': 0.8, 
        'bagging_fraction': 0.8, 
        'bagging_freq': 5,
        #'colsample_bytree': 0.3542618105439753
        #'verbose': 0
    }
    X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(x_train, y_train, test_size=0.18, random_state=i)
        
    lgb_train = lgb.Dataset(X_train_cv, y_train_cv)
    lgb_eval = lgb.Dataset(X_test_cv, y_test_cv, reference=lgb_train)

# train
    gbm = lgb.train(params,
            lgb_train,
            #num_boost_round=rnds,
            valid_sets=lgb_eval,
            verbose_eval=10,
            early_stopping_rounds=50)
    
    if denom != 0:
        pred = gbm.predict(x_test, num_iteration=gbm.best_iteration+50)      
        preds += pred
    else:
        pred = gbm.predict(x_test, num_iteration=gbm.best_iteration+50)
        preds = pred.copy()
    denom += 1
    #submission = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])
    #submission = submission.reset_index()
    #submission.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]
    #submission['ID'] = pid
    #submission.to_csv('submission_xgb_fold_'  + str(i) + '.csv', index=False)
    
preds /= denom
submission = pd.read_csv("sample_submission.csv")
submission['target'] = preds
submission.columns = ["id","target"]
submission.to_csv("sub_LGBM.csv", index=False)
#submission['ID'] = pid
#y_pred = gbm.predict(X_test_vect_lgbm, num_iteration=gbm.best_iteration)
#y_pred = np.argmax(y_pred, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:100: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.66161
[20]	valid_0's auc: 0.66414
[30]	valid_0's auc: 0.666326
[40]	valid_0's auc: 0.66858
[50]	valid_0's auc: 0.670733
[60]	valid_0's auc: 0.671681
[70]	valid_0's auc: 0.671979
[80]	valid_0's auc: 0.671796
[90]	valid_0's auc: 0.672664
[100]	valid_0's auc: 0.673816
[110]	valid_0's auc: 0.674548
[120]	valid_0's auc: 0.675141
[130]	valid_0's auc: 0.675535
[140]	valid_0's auc: 0.676134
[150]	valid_0's auc: 0.676436
[160]	valid_0's auc: 0.676786
[170]	valid_0's auc: 0.677401
[180]	valid_0's auc: 0.677213
[190]	valid_0's auc: 0.678289
[200]	valid_0's auc: 0.679207
[210]	valid_0's auc: 0.679427
[220]	valid_0's auc: 0.679715
[230]	valid_0's auc: 0.680204
[240]	valid_0's auc: 0.680367
[250]	valid_0's auc: 0.68077
[260]	valid_0's auc: 0.681463
[270]	valid_0's auc: 0.68131
[280]	valid_0's auc: 0.68155
[290]	valid_0's auc: 0.681983
[300]	valid_0's auc: 0.682351
[310]	valid_0's auc: 0.682363
[320]	valid_0's auc: 0.

In [132]:
a = X_test.columns

In [134]:
df = pd.DataFrame(a)

In [137]:
df['fi'] = [6916,  2439, 10009,  1912,  3718,  1444,  1664,  1014,   982,     5,    14,   141,
    21,   177, 10657,  1587,  1567,   693,  7187,  8825, 19999,  7036,   851,  2428,
  1815,  2682,  7255,  1533,   631,  3010,   179, 11870,  2926,  5964, 19615, 16080,
  6897,  8216,  8227,  8576,  4671,  4825,  4960,  5675,  5670,  4945,  8866,  7417,
  4764,  6144,  8612,   671,  1332,  1654,  1116,  1285,   631]

In [138]:
df

,0,fi
0,ps_ind_01,6916
1,ps_ind_02_cat,2439
2,ps_ind_03,10009
3,ps_ind_04_cat,1912
4,ps_ind_05_cat,3718
5,ps_ind_06_bin,1444
6,ps_ind_07_bin,1664
7,ps_ind_08_bin,1014
8,ps_ind_09_bin,982
9,ps_ind_10_bin,5
